In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


'''
None || 20000
'''
nb = None
nb_trains = nb
nb_valids = nb
nb_tests = nb

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [3]:
# cd drive/MyDrive/Projects/JPX_Tokyo_Stock/working

#### Load data

In [4]:
from data_utils import *

In [5]:
prices = pd.read_csv('../__input__/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv', parse_dates=['Date'], nrows = nb_trains)
sprices = pd.read_csv("../__input__/jpx-tokyo-stock-exchange-prediction/train_files/secondary_stock_prices.csv", parse_dates=['Date'], nrows = nb_trains)

supplemental_prices = pd.read_csv("../__input__/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv", parse_dates=['Date'], nrows = nb_valids)
supplemental_sprices = pd.read_csv("../__input__/jpx-tokyo-stock-exchange-prediction/supplemental_files/secondary_stock_prices.csv", parse_dates=['Date'], nrows = nb_valids)

In [6]:
hidden_test = pd.read_csv('../__input__/jpx-tokyo-stock-exchange-prediction/example_test_files/stock_prices.csv', nrows = nb_tests, parse_dates=['Date'])

#### Get CloseT1, CloseT2; remove missing value

In [7]:
def fillCloseT1T2(_df: pd.DataFrame):
    assert all([x in _df.columns for x in ['Target', 'CloseT1', 'CloseT2']])

    def calCloseT2fromT1(target, closeT1):
        '''
        target = ( close(t + 2) - close(t + 1) )/close(t + 1)
        => close(t + 2) = target * close(t + 1) + close(t + 1)
        '''
        return target * closeT1 + closeT1

    df = _df.copy()
    
    sorted_tmp = df.groupby(by=['SecuritiesCode']).apply(
        lambda df_code: fillInterpolate(
            df_code.sort_values(by=['Date'], ascending= True), 
            ['CloseT1']
        )[['SecuritiesCode', 'Date', 'CloseT1']]
    ).reset_index(drop= True)

    # left or right 
    df = df.drop('CloseT1', axis = 1).merge(
        sorted_tmp,
        how='left', on=['SecuritiesCode', 'Date']
    )

    df = df.fillna({"CloseT1": df['Close']})

    df['CloseT2'] = df.apply(lambda x: calCloseT2fromT1(x['Target'], x['CloseT1']) if np.isnan(x['CloseT2']) else x['CloseT2'], axis = 1)
    return df

In [8]:
def DataPreprocessing_for_All(_df: pd.DataFrame):
    df = _df.copy() 
    df = df.dropna(axis=0, subset=['Target'] )

    df_return = df[['Date', 'SecuritiesCode', 'Volume', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag', 'Close', 'Target']]
    df_return['ExpectedDividend'].fillna(-1, inplace= True)

    # add close of t + 1 and t + 2; use as Target
    t = df_return[['SecuritiesCode', 'Date', 'Close']]
    t.loc[:, 'Date'] = t['Date'] - pd.Timedelta("1 day")
    t.columns = t.columns.str.replace('Close', 'CloseT1')
    t = t.iloc[:-1]
    df_return = pd.merge(df_return, t, how='left', on= ['SecuritiesCode', 'Date'])

    t = df_return[['SecuritiesCode', 'Date', 'Close']]
    t.loc[:, 'Date'] = t['Date'] - pd.Timedelta("2 day")
    t.columns = t.columns.str.replace('Close', 'CloseT2')
    t = t.iloc[:-2]
    df_return = pd.merge(df_return, t, how='left', on= ['SecuritiesCode', 'Date'])

    df_return = df_return.drop('Close', axis= 1)

    sorted_tmp = df.groupby(by=['SecuritiesCode']).apply(
        lambda df_code: df_code[
            ['SecuritiesCode', 'Date', 'Open', 'High', 'Low', 'Close']
        ]
    ).reset_index(drop= True)

    df_return = df_return.merge(
        sorted_tmp,
        how='left', on=['SecuritiesCode', 'Date']
    )
    
    df_return.dropna(subset = df_return.columns.drop(['CloseT1', 'CloseT2']), axis = 0, inplace = True)    

    df_return = fillCloseT1T2(df_return)
    return df_return

In [9]:
old_merge_df = pd.concat([prices, supplemental_prices])
old_snd_merge_df = pd.concat([sprices, supplemental_sprices])

In [10]:
merge_df = DataPreprocessing_for_All(old_merge_df)
snd_merge_df = DataPreprocessing_for_All(old_snd_merge_df)

/home/kienanh/miniconda3/envs/env/lib/python3.7/site-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/kienanh/miniconda3/envs/env/lib/python3.7/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/kienanh/miniconda3/envs/env/lib/python3.7/site-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/panda

In [11]:
merge_df.columns

Index(['Date', 'SecuritiesCode', 'Volume', 'AdjustmentFactor',
       'ExpectedDividend', 'SupervisionFlag', 'Target', 'CloseT2', 'Open',
       'High', 'Low', 'Close', 'CloseT1'],
      dtype='object')

In [12]:
sorted_cols = [
    'Date', 'SecuritiesCode', 
    'ExpectedDividend',  'AdjustmentFactor', 'SupervisionFlag', 
    'Volume', 'Open', 'High', 'Low', 'Close', 
    'Target', 'CloseT1' ,  'CloseT2' 
]

In [13]:
merge_df = merge_df[sorted_cols]
snd_merge_df = snd_merge_df[sorted_cols]

#### Check DataPreprocessing

In [14]:
old_merge_df.info(null_counts= True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2562489 entries, 0 to 229957
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   RowId             2562489 non-null  object        
 1   Date              2562489 non-null  datetime64[ns]
 2   SecuritiesCode    2562489 non-null  int64         
 3   Open              2554175 non-null  float64       
 4   High              2554175 non-null  float64       
 5   Low               2554175 non-null  float64       
 6   Close             2554175 non-null  float64       
 7   Volume            2562489 non-null  int64         
 8   AdjustmentFactor  2562489 non-null  float64       
 9   ExpectedDividend  20876 non-null    float64       
 10  SupervisionFlag   2562489 non-null  bool          
 11  Target            2562245 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(7), int64(2), object(1)
memory usage: 237.0+ MB


In [15]:
merge_df.info(null_counts= True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2554169 entries, 0 to 2554168
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   Date              2554169 non-null  datetime64[ns]
 1   SecuritiesCode    2554169 non-null  int64         
 2   ExpectedDividend  2554169 non-null  float64       
 3   AdjustmentFactor  2554169 non-null  float64       
 4   SupervisionFlag   2554169 non-null  bool          
 5   Volume            2554169 non-null  int64         
 6   Open              2554169 non-null  float64       
 7   High              2554169 non-null  float64       
 8   Low               2554169 non-null  float64       
 9   Close             2554169 non-null  float64       
 10  Target            2554169 non-null  float64       
 11  CloseT1           2554169 non-null  float64       
 12  CloseT2           2554169 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(9),

In [16]:
old_merge_df[old_merge_df['SecuritiesCode'] == 1301].head(10)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1865,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920
3730,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092
5595,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100
7460,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295
9325,20170112_1301,2017-01-12,1301,2745.0,2747.0,2703.0,2731.0,28700,1.0,NaN,False,-0.006613
11190,20170113_1301,2017-01-13,1301,2707.0,2730.0,2707.0,2722.0,19400,1.0,NaN,False,-0.006657
13055,20170116_1301,2017-01-16,1301,2725.0,2725.0,2696.0,2704.0,20100,1.0,NaN,False,0.002978
14920,20170117_1301,2017-01-17,1301,2702.0,2704.0,2682.0,2686.0,18400,1.0,NaN,False,0.001856
16785,20170118_1301,2017-01-18,1301,2689.0,2695.0,2681.0,2694.0,12100,1.0,NaN,False,0.014079


In [17]:
merge_df[merge_df['SecuritiesCode'] == 1301].head(10)

,Date,SecuritiesCode,ExpectedDividend,AdjustmentFactor,SupervisionFlag,Volume,Open,High,Low,Close,Target,CloseT1,CloseT2
0,2017-01-04,1301,-1.0,1.0,False,31400,2734.0,2755.0,2730.0,2742.0,0.000730,2738.0,2740.000000
1863,2017-01-05,1301,-1.0,1.0,False,17900,2743.0,2747.0,2735.0,2738.0,0.002920,2740.0,2748.000000
3726,2017-01-06,1301,-1.0,1.0,False,19900,2734.0,2744.0,2720.0,2740.0,-0.001092,2741.0,2738.007642
5590,2017-01-10,1301,-1.0,1.0,False,24200,2745.0,2754.0,2735.0,2748.0,-0.005100,2745.0,2731.000000
7449,2017-01-11,1301,-1.0,1.0,False,9300,2748.0,2752.0,2737.0,2745.0,-0.003295,2731.0,2722.000000
9310,2017-01-12,1301,-1.0,1.0,False,28700,2745.0,2747.0,2703.0,2731.0,-0.006613,2722.0,2704.000000
11168,2017-01-13,1301,-1.0,1.0,False,19400,2707.0,2730.0,2707.0,2722.0,-0.006657,2713.0,2694.940089
13027,2017-01-16,1301,-1.0,1.0,False,20100,2725.0,2725.0,2696.0,2704.0,0.002978,2686.0,2694.000000
14884,2017-01-17,1301,-1.0,1.0,False,18400,2702.0,2704.0,2682.0,2686.0,0.001856,2694.0,2699.000000
16743,2017-01-18,1301,-1.0,1.0,False,12100,2689.0,2695.0,2681.0,2694.0,0.014079,2699.0,2737.000000


In [18]:
((merge_df['CloseT2'][merge_df['SecuritiesCode'] == 1301] - merge_df['CloseT1'][merge_df['SecuritiesCode'] == 1301])/merge_df['CloseT1'][merge_df['SecuritiesCode'] == 1301]).head(10)

0        0.000730
1863     0.002920
3726    -0.001092
5590    -0.005100
7449    -0.003295
9310    -0.006613
11168   -0.006657
13027    0.002978
14884    0.001856
16743    0.014079
dtype: float64

#### Data Split

In [19]:
def train_valid_test_split(df: pd.DataFrame):
    df_train = df[df['Date'] <= '2021-05-27'].reset_index(drop= True)

    df_valid = df[
        (df['Date'] > '2021-05-27') & (df['Date'] <= '2021-12-03')
    ].reset_index(drop= True)
    
    df_test = df[df['Date'] >= '2021-12-06'].reset_index(drop= True)
    return df_train, df_valid, df_test

In [20]:
train_df, valid_df, test_df = train_valid_test_split(merge_df)
snd_train_df, snd_valid_df, snd_test_df = train_valid_test_split(snd_merge_df)

In [21]:
len(train_df), len(valid_df), len(test_df)

(2067600, 257323, 229246)

In [22]:
train_df.to_csv('../__input__/1_DataPreprocessing/train_df.csv', index = False)
valid_df.to_csv('../__input__/1_DataPreprocessing/valid_df.csv', index = False)
test_df.to_csv('../__input__/1_DataPreprocessing/test_df.csv', index = False)

snd_train_df.to_csv('../__input__/1_DataPreprocessing/snd_train_df.csv', index = False)
snd_valid_df.to_csv('../__input__/1_DataPreprocessing/snd_valid_df.csv', index = False)
snd_test_df.to_csv('../__input__/1_DataPreprocessing/snd_test_df.csv', index = False)


#### Mini TrainSet

In [23]:
mini_train = train_df.groupby(by=['SecuritiesCode']).apply(
    lambda df_code: df_code.iloc[-3:]
).reset_index(drop= True) 
mini_train

,Date,SecuritiesCode,ExpectedDividend,AdjustmentFactor,SupervisionFlag,Volume,Open,High,Low,Close,Target,CloseT1,CloseT2
0,2021-05-25,1301,-1.0,1.0,False,23400,2961.0,2965.0,2919.0,2924.0,-0.019009,2946.0,2890.0
1,2021-05-26,1301,-1.0,1.0,False,14500,2930.0,2950.0,2919.0,2946.0,-0.007266,2890.0,2869.0
2,2021-05-27,1301,-1.0,1.0,False,36800,2929.0,2943.0,2889.0,2890.0,0.009411,2869.0,2896.0
3,2021-05-25,1332,-1.0,1.0,False,1225800,520.0,520.0,512.0,513.0,-0.013699,511.0,504.0
4,2021-05-26,1332,-1.0,1.0,False,1036900,509.0,514.0,506.0,511.0,0.019841,504.0,514.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,2021-05-26,9994,-1.0,1.0,False,4400,2281.0,2283.0,2273.0,2279.0,0.003947,2280.0,2289.0
5996,2021-05-27,9994,-1.0,1.0,False,4300,2291.0,2291.0,2280.0,2280.0,-0.004806,2289.0,2278.0
5997,2021-05-25,9997,-1.0,1.0,False,223700,1176.0,1178.0,1145.0,1146.0,-0.026978,1112.0,1082.0
5998,2021-05-26,9997,-1.0,1.0,False,210200,1140.0,1140.0,1106.0,1112.0,0.020333,1082.0,1104.0


In [24]:
mini_valid = valid_df.groupby(by=['SecuritiesCode']).apply(
    lambda df_code: df_code.iloc[-3:]
).reset_index(drop= True) 
mini_valid

,Date,SecuritiesCode,ExpectedDividend,AdjustmentFactor,SupervisionFlag,Volume,Open,High,Low,Close,Target,CloseT1,CloseT2
0,2021-12-01,1301,-1.0,1.0,False,10600,2909.0,2936.0,2909.0,2911.0,0.016706,2933.00,2982.000000
1,2021-12-02,1301,-1.0,1.0,False,15200,2949.0,2973.0,2933.0,2933.0,-0.003689,2982.00,2971.000000
2,2021-12-03,1301,-1.0,1.0,False,11400,2983.0,2983.0,2965.0,2982.0,0.031639,3002.75,3097.754544
3,2021-12-01,1332,-1.0,1.0,False,1661300,568.0,578.0,568.0,574.0,0.020942,573.00,585.000000
4,2021-12-02,1332,-1.0,1.0,False,1525800,566.0,577.0,565.0,573.0,0.006838,585.00,589.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,2021-12-02,9994,-1.0,1.0,False,9200,2340.0,2382.0,2340.0,2362.0,0.012139,2389.00,2418.000000
5996,2021-12-03,9994,-1.0,1.0,False,6500,2388.0,2396.0,2380.0,2389.0,0.009098,2401.75,2423.602151
5997,2021-12-01,9997,-1.0,1.0,False,339100,661.0,688.0,660.0,685.0,0.017544,684.00,696.000000
5998,2021-12-02,9997,-1.0,1.0,False,342900,681.0,692.0,680.0,684.0,0.014368,696.00,706.000000


In [25]:
mini_train.to_csv('../__input__/1_DataPreprocessing/mini_train.csv', index = False)
mini_valid.to_csv('../__input__/1_DataPreprocessing/mini_valid.csv', index = False)

#### Hidden Test

In [26]:
%%time
for i in range(20):
    hidden_test2 = DataPreprocessing_for_HiddenTest(hidden_test, mini_valid)

CPU times: user 2min 10s, sys: 4.43 ms, total: 2min 10s
Wall time: 2min 10s


In [27]:
%%time
hidden_test2 = DataPreprocessing_for_HiddenTest(hidden_test, mini_valid)

CPU times: user 6.85 s, sys: 3.92 ms, total: 6.86 s
Wall time: 6.85 s


In [28]:
id(hidden_test)

140179790960080

In [29]:
id(hidden_test2)

140179790444688

In [30]:
hidden_test.info(null_counts= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   RowId             4000 non-null   object        
 1   Date              4000 non-null   datetime64[ns]
 2   SecuritiesCode    4000 non-null   int64         
 3   Open              3990 non-null   float64       
 4   High              3990 non-null   float64       
 5   Low               3990 non-null   float64       
 6   Close             3990 non-null   float64       
 7   Volume            4000 non-null   int64         
 8   AdjustmentFactor  4000 non-null   float64       
 9   ExpectedDividend  0 non-null      float64       
 10  SupervisionFlag   4000 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(6), int64(2), object(1)
memory usage: 316.5+ KB


In [31]:
hidden_test2.info(null_counts= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              4000 non-null   datetime64[ns]
 1   SecuritiesCode    4000 non-null   int64         
 2   Open              4000 non-null   float64       
 3   High              4000 non-null   float64       
 4   Low               4000 non-null   float64       
 5   Close             4000 non-null   float64       
 6   Volume            4000 non-null   int64         
 7   AdjustmentFactor  4000 non-null   float64       
 8   ExpectedDividend  4000 non-null   float64       
 9   SupervisionFlag   4000 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(6), int64(2)
memory usage: 285.3 KB


In [32]:
hidden_test[np.isnan(hidden_test['Close'])]

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
26,20211206_1723,2021-12-06,1723,NaN,NaN,NaN,NaN,0,1.0,NaN,False
32,20211206_1787,2021-12-06,1787,NaN,NaN,NaN,NaN,0,1.0,NaN,False
716,20211206_4628,2021-12-06,4628,NaN,NaN,NaN,NaN,0,1.0,NaN,False
848,20211206_5217,2021-12-06,5217,NaN,NaN,NaN,NaN,0,1.0,NaN,False
922,20211206_5918,2021-12-06,5918,NaN,NaN,NaN,NaN,0,1.0,NaN,False
1931,20211206_9733,2021-12-06,9733,NaN,NaN,NaN,NaN,0,1.0,NaN,False
2101,20211207_1981,2021-12-07,1981,NaN,NaN,NaN,NaN,0,1.0,NaN,False
2255,20211207_2761,2021-12-07,2761,NaN,NaN,NaN,NaN,0,1.0,NaN,False
2716,20211207_4628,2021-12-07,4628,NaN,NaN,NaN,NaN,0,1.0,NaN,False
3931,20211207_9733,2021-12-07,9733,NaN,NaN,NaN,NaN,0,1.0,NaN,False


In [33]:
mini_valid[mini_valid['SecuritiesCode'] == 4628]

,Date,SecuritiesCode,ExpectedDividend,AdjustmentFactor,SupervisionFlag,Volume,Open,High,Low,Close,Target,CloseT1,CloseT2
2148,2021-11-30,4628,-1.0,1.0,False,500,37800.0,37800.0,37150.0,37150.0,0.010767,37433.333333,37550.000000
2149,2021-12-02,4628,-1.0,1.0,False,500,37750.0,37750.0,37350.0,37550.0,0.000000,38000.000000,38000.000000
2150,2021-12-03,4628,-1.0,1.0,False,1200,37400.0,38000.0,37400.0,38000.0,0.000000,38004.545455,38004.545455


In [34]:
hidden_test[hidden_test['SecuritiesCode'] == 4628]

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
716,20211206_4628,2021-12-06,4628,NaN,NaN,NaN,NaN,0,1.0,NaN,False
2716,20211207_4628,2021-12-07,4628,NaN,NaN,NaN,NaN,0,1.0,NaN,False


In [35]:
hidden_test2[hidden_test2['SecuritiesCode'] == 4628]

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
716,2021-12-06,4628,37400.0,38000.0,37400.0,38000.0,0,1.0,-1.0,False
2716,2021-12-07,4628,37400.0,38000.0,37400.0,38000.0,0,1.0,-1.0,False


In [36]:
hidden_test[hidden_test['SecuritiesCode'] == 1301]

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
0,20211206_1301,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False
2000,20211207_1301,2021-12-07,1301,2998.0,3065.0,2990.0,3065.0,19100,1.0,NaN,False


In [37]:
hidden_test2[hidden_test2['SecuritiesCode'] == 1301]

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
0,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,-1.0,False
2000,2021-12-07,1301,2998.0,3065.0,2990.0,3065.0,19100,1.0,-1.0,False


In [38]:
hidden_test2.head(20)

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
0,2021-12-06,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,-1.0,False
1,2021-12-06,1332,592.0,599.0,588.0,589.0,1360800,1.0,-1.0,False
2,2021-12-06,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,-1.0,False
3,2021-12-06,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,-1.0,False
4,2021-12-06,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,-1.0,False
5,2021-12-06,1377,3185.0,3215.0,3180.0,3200.0,77000,1.0,-1.0,False
6,2021-12-06,1379,1890.0,1898.0,1885.0,1889.0,48500,1.0,-1.0,False
7,2021-12-06,1381,3200.0,3200.0,3150.0,3150.0,1600,1.0,-1.0,False
8,2021-12-06,1407,6100.0,6290.0,6010.0,6220.0,246700,1.0,-1.0,False
9,2021-12-06,1413,2185.0,2211.0,2181.0,2190.0,9200,1.0,-1.0,False


In [39]:
valid_df[valid_df['SecuritiesCode'] == 9733]

,Date,SecuritiesCode,ExpectedDividend,AdjustmentFactor,SupervisionFlag,Volume,Open,High,Low,Close,Target,CloseT1,CloseT2
1924,2021-05-28,9733,-1.0,1.0,False,100,5340.0,5340.0,5340.0,5340.0,0.000000,5430.000000,5430.000000
3921,2021-05-31,9733,-1.0,1.0,False,100,5380.0,5380.0,5380.0,5380.0,0.000000,5700.000000,5700.000000
9907,2021-06-03,9733,-1.0,1.0,False,2000,6380.0,6380.0,5580.0,5870.0,0.000000,5970.000000,5970.000000
11902,2021-06-04,9733,-1.0,1.0,False,1200,5870.0,5970.0,5870.0,5970.0,-0.067002,5860.000000,5467.370184
15891,2021-06-08,9733,-1.0,1.0,False,200,5610.0,5610.0,5570.0,5570.0,-0.007380,5420.000000,5380.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229335,2021-11-12,9733,-1.0,1.0,False,500,5590.0,5590.0,5520.0,5590.0,0.000000,5621.428571,5621.428571
241294,2021-11-22,9733,-1.0,1.0,False,300,5550.0,5600.0,5520.0,5520.0,0.000000,5650.000000,5650.000000
247276,2021-11-26,9733,-1.0,1.0,False,100,5530.0,5530.0,5530.0,5530.0,0.012500,5661.428571,5732.196429
249274,2021-11-29,9733,-1.0,1.0,False,300,5530.0,5600.0,5530.0,5600.0,0.000000,5670.000000,5670.000000


In [40]:
hidden_test[hidden_test['SecuritiesCode'] == 9733]

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
1931,20211206_9733,2021-12-06,9733,NaN,NaN,NaN,NaN,0,1.0,NaN,False
3931,20211207_9733,2021-12-07,9733,NaN,NaN,NaN,NaN,0,1.0,NaN,False


In [41]:
hidden_test2[hidden_test2['SecuritiesCode'] == 9733]

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
1931,2021-12-06,9733,5600.0,5670.0,5600.0,5670.0,0,1.0,-1.0,False
3931,2021-12-07,9733,5600.0,5670.0,5600.0,5670.0,0,1.0,-1.0,False
